In [ ]:
import os
import kagglehub
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

# Import training functions from train.py
from train import train_model, plot_metrics

# Download latest version of the dataset from Kaggle
path = kagglehub.dataset_download("tongpython/cat-and-dog")


### Create train and test datset with dataloader

In [ ]:
# Set subclass of Dataset for kaggle Dataset
class PetDataSet(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data = ImageFolder(data_dir, transform=transform)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]
    
    @property
    def classes(self):
        return self.data.classes
    
# Set paths to training and test folders with classes as subdirs
train_folder = os.path.join(path, "training_set/training_set")
test_folder = os.path.join(path, "test_set/test_set")

# Get a dictionary associating target values with folder names
target_to_class = {v: k for k, v in ImageFolder(train_folder).class_to_idx.items()}
print("Classes found:", target_to_class)

# Define transformations and create DataLoaders
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

train_dataset = PetDataSet(train_folder, transform=transform)
test_dataset = PetDataSet(test_folder, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

### Train pretrained model

In [ ]:
train_losses, test_losses, accuracies = train_model(train_loader, 
                                                    test_loader,
                                                    num_epochs=10, 
                                                    learning_rate=0.001, 
                                                    model_name="pretrained_classifier",
                                                    pretrained=True,
                                                    use_LR_scheduler=True)
plot_metrics(train_losses, test_losses, accuracies)

Training loop:  44%|████▍     | 56/126 [00:21<00:26,  2.65it/s]

## Train model from scratch

In [ ]:
train_losses, test_losses, accuracies = train_model(train_loader, 
                                                    test_loader, 
                                                    num_epochs=25, 
                                                    learning_rate=0.001, 
                                                    model_name="scratch_classifier", 
                                                    pretrained=False,
                                                    use_LR_scheduler=True)
plot_metrics(train_losses, test_losses, accuracies)